In [1]:
import os

from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd

import jams
import numpy as np

import muda

In [2]:
def root(x):
    return os.path.splitext(os.path.basename(x))[0]

AUDIO = jams.util.find_with_extension('/home/bmcfee/data/eric_chords/audio/', 'mp3')
ANNOS = jams.util.find_with_extension('/home/bmcfee/data/eric_chords/references_v2/', 'jams')

# Make sure there are the same number of files
assert len(AUDIO) == len(ANNOS)

# And that they're in agreement
assert all([root(_1) == root(_2) for (_1, _2) in zip(AUDIO, ANNOS)])

In [3]:
OUTDIR = '/home/bmcfee/working/chords/augmentation/'

In [4]:
!mkdir -p {OUTDIR}

In [5]:
def augment(afile, jfile, deformer, outpath):
    
    jam = muda.load_jam_audio(jfile, afile)
    
    base = root(afile)
    outfile = os.path.join(outpath, base)
    
    for i, jam_out in enumerate(deformer.transform(jam)):
        muda.save('{}.{}.flac'.format(outfile, i),
                  '{}.{}.jams'.format(outfile, i),
                 jam_out, strict=False)

In [6]:
# Create the augmentation engine
pitcher = muda.deformers.PitchShift(n_semitones=[-2, -1, 1, 2])

In [7]:
with open('/home/bmcfee/working/chords/muda.pkl', 'wb') as fd:
    pickle.dump(pitcher, fd)

In [8]:
Parallel(n_jobs=20, verbose=10)(delayed(augment)(aud, jam, pitcher, OUTDIR) for (aud, jam) in zip(AUDIO, ANNOS));

[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   18.6s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   23.7s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:   40.3s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:   55.4s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:  1.3min
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:  1.5min
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:  1.8min
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:  2.2min
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:  2.6min
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:  3.0min
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:  3.4min
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:  3.9min
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:  4.4min
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:  4.8min
[Parallel(n_jobs=20)]: Done 225 tasks      | elapsed:  5.4min
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed:  5.9min
[Paralle